In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e11/sample_submission.csv
/kaggle/input/playground-series-s3e11/train.csv
/kaggle/input/playground-series-s3e11/test.csv


In [2]:
import seaborn as sns

In [3]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmplofk9b4r
  JVM stdout: /tmp/tmplofk9b4r/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmplofk9b4r/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,1 month and 16 days
H2O_cluster_name:,H2O_from_python_unknownUser_7c880q
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [4]:
data = h2o.import_file('/kaggle/input/playground-series-s3e11/train.csv')
test =h2o.import_file('/kaggle/input/playground-series-s3e11/test.csv')
display(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


id,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
0,8.61,3,2,2,2,10.3,1,0,32,36509,0,0,0,0,0,62.09
1,5,2,4,0,3,6.66,1,0,1,28206,1,0,0,0,0,121.8
2,14.08,4,0,0,3,21.3,1,0,26,21215,1,0,0,0,0,83.51
3,4.02,3,5,0,0,14.8,0,1,36,21215,1,0,0,0,0,66.78
4,2.13,3,5,0,3,17,1,1,20,27694,1,1,1,1,1,111.51
5,9.08,4,5,5,3,7.26,0,1,5,33858,1,0,1,1,1,142.58
6,4.8,2,1,0,2,9.58,0,0,6,27694,1,1,1,1,1,69.47
7,4.29,3,2,0,2,16.9,1,0,2,23688,1,1,1,1,1,68.84
8,8.55,3,5,0,2,13.8,1,0,6,38382,0,0,0,0,0,87.07
9,3.08,4,1,0,3,15.7,1,1,9,27694,1,1,1,1,1,80.29


In [5]:
train_col = data.columns
train_col.remove('cost')

In [6]:
dataset_split = data.split_frame(ratios = [0.8], seed = 123456)
train = dataset_split[0]
val = dataset_split[1]

In [7]:
aml = H2OAutoML(max_models = 20, max_runtime_secs = 200, seed = 1, sort_metric = "deviance")

In [8]:
aml.train(x = train_col, y = 'cost', training_frame = train, validation_frame = val)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,55.0,55.0,316526.0,0.0,10.0,7.4545455,1.0,798.0,454.07272
,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2023-04-26 13:08:05,0.003 sec,0.0,29.9445699,25.9795804,896.6772648,29.9789960,25.9942258,898.7402034
,2023-04-26 13:08:06,1.265 sec,5.0,28.9105778,25.0508026,835.8215096,29.0803660,25.1802166,845.6676888
,2023-04-26 13:08:08,2.418 sec,10.0,28.3627553,24.4904726,804.4458854,28.6586514,24.7325463,821.3183017
,2023-04-26 13:08:08,3.312 sec,15.0,28.0376663,24.1296306,786.1107289,28.4620068,24.4861507,810.0858304
,2023-04-26 13:08:09,4.297 sec,20.0,27.7923885,23.8285766,772.4168594,28.3363270,24.2934333,802.9474288
,2023-04-26 13:08:10,5.171 sec,25.0,27.6020814,23.6031264,761.8748976,28.2573163,24.1667762,798.4759231
,2023-04-26 13:08:11,6.145 sec,30.0,27.4472231,23.4217719,753.3500568,28.2078140,24.0782397,795.6807691
,2023-04-26 13:08:12,7.016 sec,35.0,27.3321012,23.2972354,747.0437550,28.1930604,24.0419816,794.8486548


In [9]:
print(aml.leader)

Model Details
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_1_20230426_130541


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    55                 55                          316526                 0            10           7.45455       1             798           454.073

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 741.3105458415205
RMSE: 27.22701867339721
MAE: 23.189565987724293
RMSLE: 0.29173854981662667
Mean Residual Deviance: 741.3105458415205

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 794.517429037482
RMSE: 28.18718554658272
MAE: 24.020080447957493
RMSLE: 0.30179966000069386
Mean Residual Deviance: 794.517429037482

Scori

In [10]:
pred = aml.predict(test)
pred.head()

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict
102.149
98.8605
98.0534
105.104
89.3165
102.136
106.912
97.382
89.5616
91.202


In [11]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e11/sample_submission.csv')
sample_submission.shape

(240224, 2)

In [12]:
sample_submission['cost'] = pred.as_data_frame().values
sample_submission.to_csv('h2o_automl_submission.csv', index=False)